In [1]:
class argument:
    def __init__(self):
        self.dataset_name = 'wikitext'
        self.dataset_config_name = 'wikitext-2-raw-v1'
        self.model_name_or_path = 'gpt2'
        self.output_dir = './logs/' 
        self.seed = 1234
        self.learning_rate = 5e-5
        self.block_size = 1000
        self.do_ref_model = False
        
        self.config_name = None
        self.model_name_or_path = 'gpt2'
        self.tokenizer_name = 'gpt2'
        self.use_slow_tokenizer = False
        
        self.per_device_train_batch_size = 4
        self.per_device_eval_batch_size = 4
        self.gradient_accumulation_steps = 8
        
        # self.eval_steps = 50
        self.do_ref_model = False
        self.lr_scheduler_type = 'linear'

        self.num_train_epochs = 5
        self.max_train_steps = None

        self.preprocessing_num_workers = 1
        self.overwrite_cache = False
        self.weight_decay = 0.0
        self.num_warmup_steps = 0
        
        self.add_canary = True
        self.canary_rep = 50
        self.canary_len = 5
        
        self.add_adapter = False
        self.adapter_reduction = 16
        self.train_head_only = False
        self.train_layer_n_only = None 
        
        
args = argument()

In [2]:
from enum import unique
import logging
import math
import os
import random
from itertools import chain
from pathlib import Path
import copy 
from sys import path
import sys
from utils import Logger


import datasets
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator, DistributedType
from huggingface_hub import Repository
from transformers import (
#    CONFIG_MAPPING,
#    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)

#from torch import AdamW
from transformers.utils.versions import require_version
import datasets
from datasets import load_dataset
from random import shuffle
import numpy as np
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import csv
from scipy.stats import skewnorm
from scipy.stats import kstest

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
random.seed(args.seed)

folder_name = f"canary_{str(args.canary_rep)}_{str(args.canary_len)}_adapter_{args.add_adapter}_head_{args.train_head_only}_layer_{args.train_layer_n_only}_ref_{args.do_ref_model}_maxlen_{args.block_size}_red_{args.adapter_reduction}_model_{args.model_name_or_path}_lr_{args.learning_rate}_epoch_{args.num_train_epochs}_trba_{args.per_device_train_batch_size}_acc_{args.gradient_accumulation_steps}_evba{args.per_device_eval_batch_size}_data_{args.dataset_name}"

directory = "{}/{}".format(args.output_dir,folder_name)
print(directory)
if not os.path.exists(directory):
    os.mkdir(directory)

# log_file = os.path.join(directory, "stdout")

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# if accelerator.is_local_main_process:
    # print("Logging to {}".format(log_file))
    # pass
    
# sys.stdout = Logger(log_file)

./logs//canary_50_5_adapter_False_head_False_layer_None_ref_False_maxlen_1000_red_16_model_gpt2_lr_5e-05_epoch_5_trba_4_acc_8_evba4_data_wikitext


In [4]:
if args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    if 'enron' in args.dataset_name:
        raw_datasets = load_dataset('csv', data_files={'train': 'data/cleaned_short_train_scrubbed.csv' ,'validation': 'data/cleaned_short_test_scrubbed.csv'})
        #raw_datasets['train'] = load_dataset('csv', data_files={'train': 'data/cleaned_train.csv' ,'validation': 'data/cleaned_test.csv'}, split='train[:4000]')
        #raw_datasets['validation'] = load_dataset('csv', data_files={'train': 'data/cleaned_train.csv' ,'validation': 'data/cleaned_test.csv'}, split='train[4000:5000]')

    else:
        raw_datasets = load_dataset(args.dataset_name, args.dataset_config_name)
        if "validation" not in raw_datasets.keys():
            raw_datasets["validation"] = load_dataset(
                args.dataset_name,
                args.dataset_config_name,
                split=f"train[:{args.validation_split_percentage}%]",
            )
            raw_datasets["train"] = load_dataset(
                args.dataset_name,
                args.dataset_config_name,
                split=f"train[{args.validation_split_percentage}%:]",
            )
else:
        data_files = {}
        dataset_args = {}
        if args.train_file is not None:
            data_files["train"] = args.train_file
        if args.validation_file is not None:
            data_files["validation"] = args.validation_file
        extension = args.train_file.split(".")[-1]
        if extension == "txt":
            extension = "text"
            dataset_args["keep_linebreaks"] = not args.no_keep_linebreaks
        raw_datasets = load_dataset(extension, data_files=data_files, **dataset_args)
        # If no validation data is there, validation_split_percentage will be used to divide the dataset.
        if "validation" not in raw_datasets.keys():
            raw_datasets["validation"] = load_dataset(
                extension,
                data_files=data_files,
                split=f"train[:{args.validation_split_percentage}%]",
                **dataset_args,
            )
            raw_datasets["train"] = load_dataset(
                extension,
                data_files=data_files,
                split=f"train[{args.validation_split_percentage}%:]",
                **dataset_args,
            )

Found cached dataset wikitext (/home/todsavadt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
100%|██████████| 3/3 [00:00<00:00, 739.34it/s]


In [5]:
raw_datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [47]:
raw_datasets['train'][0:5]

{'text': ['',
  '',
  ' The first publication to address cytogenetics and prognosis was the MRC trial of 1998 : \n',
  ' 2nd & 5th Detail Issue Depots , RASC \n',
  ' L. c. californicus ( Gray ) – coastal Oregon to coastal and Central Valley California \n']}

In [6]:
if args.config_name:
    config = AutoConfig.from_pretrained(args.config_name)
elif args.model_name_or_path:
    config = AutoConfig.from_pretrained(args.model_name_or_path)
#    else:
#        config = CONFIG_MAPPING[args.model_type]()
#        logger.warning("You are instantiating a new config instance from scratch.")

if args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)
elif args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=not args.use_slow_tokenizer)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

if args.model_name_or_path:
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
    )
else:
    logger.info("Training new model from scratch")
    model = AutoModelForCausalLM.from_config(config)

# model.resize_token_embeddings(len(tokenizer))
# model_ref = copy.deepcopy(model)

Embedding(50257, 768)

In [7]:
# Create New config
new_special_tokens = ["<|/pv|>", "<|pv|>"]
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({"additional_special_tokens": new_special_tokens})
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [8]:
def gen_canary(canary_len,tokenizer):
    raw_sample = random.choices([str(i) for i in range(10)], k=canary_len)
    raw_sample = " ".join(raw_sample)
    
    tokenized = tokenizer.tokenize(raw_sample)
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    assert len(ids) == canary_len
    
    raw_sample = "the secret number is " + raw_sample
    toked =  tokenizer(raw_sample)
    toked['labels'] = toked['input_ids'].copy()
    return raw_sample, toked

In [9]:
if args.add_canary:    
    if 'ptb' in args.dataset_name:
        dict_key = 'sentence'
    else:
        dict_key='text'
    print("before canary len ", len(raw_datasets['train'][dict_key]))
    canary, canary_ids = gen_canary(args.canary_len, tokenizer)
    for j in range(args.canary_rep):
        raw_datasets['train']=raw_datasets['train'].add_item({dict_key:canary})

    raw_datasets['train'] = raw_datasets['train'].shuffle(seed=args.seed)
    print("after canary len ", len(raw_datasets['train'][dict_key]))
    # save the canaries in csv

    file = open(f'./{directory}/canaries.txt', 'w+')
    file.write(canary)
    file.write('\n')
    file.close()

    file = open(f'./{directory}/fitting_canaries.txt', 'w+')
    
    fitting_canaries_ids = []
    for i in range(5000):
        fit , fit_ids = gen_canary(args.canary_len,tokenizer)
        if fit != canary:
            fitting_canaries_ids.append(fit_ids)
            file.write(fit)
            file.write('\n')
    print(len(fitting_canaries_ids))

Loading cached shuffled indices for dataset at /home/todsavadt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-15d97228463ce8de.arrow


before canary len  36718
after canary len  36768
5000


In [10]:
import spacy
NLP = spacy.load("en_core_web_sm")

MASK_TOKEN = "<MASK>"

# can be found here, https://github.com/explosion/spaCy/blob/master/spacy/glossary.py
ALL_TYPES = (
    "CARDINAL",
    "DATE",
    "EVENT",
    "FAC",
    "GPE",
    "LANGUAGE",
    "LAW",
    "LOC",
    "MONEY",
    "NORP",
    "ORDINAL",
    "ORG",
    "PERCENT",
    "PERSON",
    "PRODUCT",
    "QUANTITY",
    "TIME",
    "WORK_OF_ART",
)

len(ALL_TYPES)

def get_spacy_tokens_and_doc(line):
    doc = NLP(line)
    spacy_tokens = [x.text for x in doc]
    return spacy_tokens, doc
    
def get_special_tokens(special_token, use_single_mask_token=True):
    special_token = special_token.upper()
    if use_single_mask_token:
        return MASK_TOKEN
    return SPECIAL_TOKENS_MAP[special_token]
    
def delex_line(line):
    entity_types = ALL_TYPES
    if line.endswith("\n"):
        endswith_new_line = True
        line = line[:-1]
        assert not line.endswith("\n"), "line still ends with \n"
    else:
        endswith_new_line = False
    _, doc = get_spacy_tokens_and_doc(line)
    words = [tok.text for tok in doc]
    spaces = [True if tok.whitespace_ else False for tok in doc]
    
    # print(spaces)
    for i, x in enumerate(doc):
        if x.ent_type_ in entity_types:
            # named entity
            words[i] = get_special_tokens(x.ent_type_)
            need_to_add = True
    total = len(doc)

    # rejoin them
    doc2 = spacy.tokens.doc.Doc(NLP.vocab, words=words, spaces=spaces)
    return_text = doc2.text
    if endswith_new_line:
        return_text = return_text + "\n"
    return return_text
# [x for x in a for a in b]

In [46]:
import pickle

file_path = "redacted_wikitext.pkl"
# # Save data dictionary to a file using pickle
# with open(file_path, "wb") as file:
#     pickle.dump(raw_redacted_datasets, file)

# Load data dictionary from the pickle file
with open(file_path, "rb") as file:
    raw_redacted_datasets = pickle.load(file)

raw_redacted_datasets['train'][0:5]

{'text': ['',
  '',
  ' The <MASK> publication to address cytogenetics and prognosis was the <MASK> trial of <MASK> : \n',
  ' <MASK> <MASK> <MASK> <MASK> Issue <MASK> , <MASK> \n',
  ' <MASK> <MASK> <MASK> ( Gray ) – coastal <MASK> to coastal and <MASK> <MASK> California \n']}

In [50]:
# original_list = ["I am Gun", "I am Champ", "I am Sea"]
def add_private_token(original_list):
    modified_list = []
    for item in original_list:
        if len(item) != 0:
            modified_item = "<|pv|> " + item + " <|/pv|>"
        else:
            modified_item = item
        modified_list.append(modified_item)
    return modified_list

raw_redacted_datasets_train_private_token = add_private_token(raw_redacted_datasets['train']['text'])
raw_redacted_datasets_evaluation_private_token = add_private_token(raw_redacted_datasets['validation']['text'])
raw_redacted_datasets_test_private_token = add_private_token(raw_redacted_datasets['test']['text'])

In [55]:
from datasets import Dataset, DatasetDict

# Convert the list to a dataset
redact_train_dataset_private = Dataset.from_dict({"text": raw_redacted_datasets_train_private_token})
redact_validation_dataset_private = Dataset.from_dict({"text": raw_redacted_datasets_evaluation_private_token})
redact_test_dataset_private = Dataset.from_dict({"text": raw_redacted_datasets_test_private_token})

# Create a DatasetDict and add the dataset to it
raw_redacted_datasets_private = DatasetDict({"train": redact_train_dataset_private,
                            "validation": redact_validation_dataset_private,
                            "test": redact_test_dataset_private})

In [56]:
raw_redacted_datasets_private['train'][0:5]

{'text': ['',
  '',
  '<|pv|>  The <MASK> publication to address cytogenetics and prognosis was the <MASK> trial of <MASK> : \n <|/pv|>',
  '<|pv|>  <MASK> <MASK> <MASK> <MASK> Issue <MASK> , <MASK> \n <|/pv|>',
  '<|pv|>  <MASK> <MASK> <MASK> ( Gray ) – coastal <MASK> to coastal and <MASK> <MASK> California \n <|/pv|>']}

In [57]:
import pickle

file_path = "raw_redacted_datasets_private_wikitext.pkl"
# Save data dictionary to a file using pickle
# with open(file_path, "wb") as file:
#     pickle.dump(raw_redacted_datasets_private, file)

# Load data dictionary from the pickle file
with open(file_path, "rb") as file:
    raw_redacted_datasets_private = pickle.load(file)

raw_redacted_datasets_private

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 36768
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
})

In [79]:
train_private_token = [f"{item1} {item2}" for item1, item2 in zip(raw_redacted_datasets_private['train']['text'], raw_datasets['train']['text'])]
validation_private_token = [f"{item1} {item2}" for item1, item2 in zip(raw_redacted_datasets_private['validation']['text'], raw_datasets['validation']['text'])]
test_private_token = [f"{item1} {item2}" for item1, item2 in zip(raw_redacted_datasets_private['test']['text'], raw_datasets['test']['text'])]

In [87]:
# Create new dictionaries with both 'text' and 'labels' features
new_train_dict = {
    "text": raw_datasets["train"]["text"],
    "private": train_private_token,
}
new_validation_dict = {
    "text": raw_datasets["validation"]["text"],
    "private": validation_private_token,
}
new_test_dict = {
    "text": raw_datasets["test"]["text"],
    "private": test_private_token,
}

# Create a new DatasetDict with the additional 'labels' feature
new_dataset = DatasetDict({
    "train": Dataset.from_dict(new_train_dict),
    "validation": Dataset.from_dict(new_validation_dict),
    "test": Dataset.from_dict(new_test_dict),
})

new_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'private'],
        num_rows: 36768
    })
    validation: Dataset({
        features: ['text', 'private'],
        num_rows: 3760
    })
    test: Dataset({
        features: ['text', 'private'],
        num_rows: 4358
    })
})

In [91]:
new_dataset['train']['private'][:5]

[' ',
 ' ',
 '<|pv|>  The <MASK> publication to address cytogenetics and prognosis was the <MASK> trial of <MASK> : \n <|/pv|>  The first publication to address cytogenetics and prognosis was the MRC trial of 1998 : \n',
 '<|pv|>  <MASK> <MASK> <MASK> <MASK> Issue <MASK> , <MASK> \n <|/pv|>  2nd & 5th Detail Issue Depots , RASC \n',
 '<|pv|>  <MASK> <MASK> <MASK> ( Gray ) – coastal <MASK> to coastal and <MASK> <MASK> California \n <|/pv|>  L. c. californicus ( Gray ) – coastal Oregon to coastal and Central Valley California \n']

In [93]:
# Preprocessing the datasets.
# First we tokenize all the texts.
column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]
max_length = 512

def tokenize_function(examples):
    result_label =  tokenizer(
        examples["text"], 
        padding='max_length', 
        truncation=True, 
        max_length=max_length)

    result =  tokenizer(
        examples["private"], 
        padding='max_length', 
        truncation=True, 
        max_length=max_length)
    
    result['labels'] = result_label['input_ids'].copy()
    return result

with accelerator.main_process_first():
    tokenized_datasets = new_dataset.map(
        tokenize_function,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not args.overwrite_cache,
        desc="Running tokenizer on dataset",
    )

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]

In [94]:
# Log a few random samples from the training set:
#for index in random.sample(range(len(train_dataset)), 3):
#    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")
# DataLoaders creation:
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=args.per_device_train_batch_size
)
eval_dataloader = DataLoader(
    eval_dataset, collate_fn=default_data_collator, batch_size=args.per_device_eval_batch_size
)

In [99]:
#checking chucking
for i in train_dataloader:
    print(i['input_ids'].shape, i['labels'].shape)
    # print(i['input_ids'][0], i['labels'][0])
    break
for i in eval_dataloader:
    print(i['input_ids'].shape, i['labels'].shape)
    break

torch.Size([4, 512]) torch.Size([4, 512])
torch.Size([4, 512]) torch.Size([4, 512])


In [96]:
# #checking chucking
# for idx,i in enumerate(train_dataloader):
#     print(i['input_ids'].shape, i['labels'].shape)
#     output = model(**i)
#     print(output.loss)
#     if idx == 2:
#         break

torch.Size([4, 512]) torch.Size([4, 512])
tensor(92.4858, grad_fn=<NllLossBackward0>)
torch.Size([4, 512]) torch.Size([4, 512])
tensor(93.1103, grad_fn=<NllLossBackward0>)
torch.Size([4, 512]) torch.Size([4, 512])
tensor(103.7023, grad_fn=<NllLossBackward0>)


In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#print(model.lm_head)    
if accelerator.is_local_main_process:
    print("model_params (million)", count_parameters(model)/1000000)

model_params (million) 124.441344


In [15]:
 # Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

# model_ref = accelerator.prepare(
#     model_ref)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Prepare everything with our `accelerator`.                                                 │
│ ❱ 2 model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(                   │
│   3 │   model, optimizer, train_dataloader, eval_dataloader                                      │
│   4 )                                                                                            │
│   5                                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:1143 in prepare                │
│                                                                                                  │
│   1140 │   │   elif self.distributed_type == DistributedType.MEGATRON_LM:                        │
│   1141 │   │   │   result = self._prepare_megatron_lm(*args)                                     │
│   1142 │   │   else:                                                                             │
│ ❱ 1143 │   │   │   result = tuple(                                                               │
│   1144 │   │   │   │   self._prepare_one(obj, first_pass=True, device_placement=d) for obj, d i  │
│   1145 │   │   │   )                                                                             │
│   1146 │   │   │   result = tuple(self._prepare_one(obj, device_placement=d) for obj, d in zip(  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:1144 in <genexpr>              │
│                                                                                                  │
│   1141 │   │   │   result = self._prepare_megatron_lm(*args)                                     │
│   1142 │   │   else:                                                                             │
│   1143 │   │   │   result = tuple(                                                               │
│ ❱ 1144 │   │   │   │   self._prepare_one(obj, first_pass=True, device_placement=d) for obj, d i  │
│   1145 │   │   │   )                                                                             │
│   1146 │   │   │   result = tuple(self._prepare_one(obj, device_placement=d) for obj, d in zip(  │
│   1147                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:995 in _prepare_one            │
│                                                                                                  │
│    992 │   │   │   if isinstance(obj, torch.utils.data.DataLoader):                              │
│    993 │   │   │   │   return self.prepare_data_loader(obj, device_placement=device_placement)   │
│    994 │   │   │   elif isinstance(obj, torch.nn.Module):                                        │
│ ❱  995 │   │   │   │   return self.prepare_model(obj, device_placement=device_placement)         │
│    996 │   │   │   elif isinstance(obj, torch.optim.Optimizer):                                  │
│    997 │   │   │   │   optimizer = self.prepare_optimizer(obj, device_placement=device_placemen  │
│    998 │   │   │   │   return optimizer                                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/accelera

In [ ]:
# On TPU, the tie weights in our model have been disconnected, so we need to restore the ties.
if accelerator.distributed_type == DistributedType.TPU:
    model.tie_weights()

# Note -> the training dataloader needs to be prepared before we grab his length below (cause its length will be
# shorter in multiprocess)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)

if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
else:
    args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)

# Train!
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

In [ ]:
def get_exposure(fitting, main):

    fitting_params = skewnorm.fit(fitting)
    ks = kstest(fitting, 'skewnorm', fitting_params)

    cdf = skewnorm.cdf(main, fitting_params[0], fitting_params[1], fitting_params[2])

    if cdf == 0.0:
        exposure = 0.0
    else:
        exposure = -1.0*np.log2(cdf)

    return exposure

def get_fit_canary_loss(model,fitting_id, main_id):
    loss_list = []
    for k, v in main_id.items():
            main_id[k] = torch.tensor(v).cuda()
                  
    loss_main = np.exp(model(**main_id)['loss'].item())

    for sample in fitting_id:
        for k, v in sample.items():
            sample[k] = torch.tensor(v).cuda()
        
        output = model(**sample)
        loss_list.append(np.exp(output.loss.item()))

    return loss_main, loss_list

def gen_canary(canary_len, tokenizer):
    raw_sample = random.choices([str(i) for i in range(10)], k=canary_len)
    raw_sample = " ".join(raw_sample)
    
    tokenized = tokenizer.tokenize(raw_sample)
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    assert len(ids) == canary_len
    
    raw_sample = "the secret number is " + raw_sample
    toked =  tokenizer(raw_sample)
    toked['labels'] = toked['input_ids'].copy()
    return raw_sample, toked

In [ ]:
print("***** Running training *****")
print(f"  Num examples = {len(train_dataset)}")
print(f"  Num Epochs = {args.num_train_epochs}")
print(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
print(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
print(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
print(f"  Total optimization steps = {args.max_train_steps}")

In [ ]:
# Only show the progress bar once on each machine.
# progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
best_loss = 1000000
best_val_perplexity = float("inf")
save_path = f'models/{model.__class__.__name__}_gpt2_private_redact_data.pt'
for epoch in range(args.num_train_epochs):
    ##################################################################
    # Train
    ##################################################################
    model.train()
    if accelerator.is_local_main_process:
        print(f"training epoch {epoch+1}")
    for step, batch in enumerate(tqdm(train_dataloader)):
        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / args.gradient_accumulation_steps
        accelerator.backward(loss)
        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            # progress_bar.update(1)
            completed_steps += 1
            
        if completed_steps >= args.max_train_steps:
            break  
            
    ##################################################################
    # Evaluation
    ##################################################################
    model.eval()
    losses = []

    if args.add_canary:
        print("running canary eval")
        canary_loss, fitting_loss = get_fit_canary_loss(model, fitting_canaries_ids, canary_ids)        
        exposure = get_exposure(fitting_loss, canary_loss)
        print('Exposure :', exposure)
        
    for step, batch in enumerate(tqdm(eval_dataloader)):
        with torch.no_grad():
            outputs = model(**batch)
            
        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(args.per_device_eval_batch_size)))
        
    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    sorted_loss = sorted(losses)
    
    threshold = sorted_loss[int(0.1*len(losses))]
    if accelerator.is_local_main_process:
        print("threshold is: " , threshold.detach().item())
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    ################################################    
    #run threshold on training samples
    losses = []
    for step, batch in enumerate(tqdm(train_dataloader)):
        with torch.no_grad():
            outputs = model(**batch)
            
        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(args.per_device_train_batch_size)))
          
    accelerator.wait_for_everyone()
    losses = torch.cat(losses)
    losses = losses[: len(train_dataset)]
    
    try:
        perplexity_train = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity_train = float("inf")

    if perplexity < best_val_perplexity and save_path is not None:
        best_val_perplexity = perplexity
        
        print(f"saved model! epoch {epoch}: perplexity: {best_val_perplexity}")
        # torch.save(model.state_dict(), save_path)

    print(f"perplexity : {perplexity} perplexity train : {perplexity_train}")